# Matrix Functions via Real Hermite/CH Systems

This notebook demonstrates how to compute symbolic matrix functions $A^n$, $\exp(tA)$, and the Drazin inverse using real Hermite systems and the Cayley-Hamilton approach. All results are displayed with LaTeX/MathJax for readability.

The theory behind this can be found at the following paper.

[Computing the Minimum Polynomial, the Function and the Drazin Inverse of a Matrix with Matlab](https://journalajrcos.com/index.php/AJRCOS/article/view/433)

## Cell 1: Library definitions
We define helper functions to compute minimal polynomials, factor them, split real and complex roots, build the Hermite system, and evaluate matrix functions symbolically.

In [1]:
# ===== Colab Cell 1: Library (real-only Hermite/CH with symbolic n, t) =====
import sympy as sp
x = sp.Symbol('x')
I = sp.I

# ---------- helpers ----------
def vec_col_major(M):
    M = sp.Matrix(M)
    n, m = M.shape
    return sp.Matrix(n*m, 1, lambda i, _: M[i % n, i // n])

def evaluate_on_matrix(A, coeffs):
    A = sp.Matrix(A)
    m = A.shape[0]
    res = sp.zeros(m)
    P = sp.eye(m)
    for k, ak in enumerate(coeffs):
        if k == 0:
            res += ak * sp.eye(m)
        else:
            P = P * A
            res += ak * P
    return sp.simplify(res)

# ---------- minimal polynomial via linear system ----------
def minimal_polynomial_via_linear(A):
    A = sp.Matrix(A)
    n = A.shape[0]
    for q in range(1, n+1):
        B_cols = [vec_col_major(A**k) for k in range(q)]
        B = sp.Matrix.hstack(*B_cols)
        b = vec_col_major(A**q)
        sol = sp.linsolve((B, -b))
        if sol != sp.EmptySet:
            c = list(next(iter(sol)))
            poly = x**q + sum(c[k] * x**k for k in range(q))
            return sp.poly(sp.factor(poly), x).monic()
    try:
        expr = A.minimal_polynomial(x)
        return sp.poly(expr, x).monic()
    except Exception:
        return A.charpoly(x).as_poly().monic()

def factor_minpoly(mA):
    expr = sp.factor(mA.as_expr())
    _, plist = sp.factor_list(expr)
    roots, mults = [], []
    for p, e in plist:
        inner = sp.roots(p)
        for lam, mr in inner.items():
            roots.append(sp.nsimplify(lam))
            mults.append(e * mr)
    return roots, mults

def split_real_conj(roots, mults):
    used = [False]*len(roots)
    reals, pairs, fallback = [], [], False
    for i, (ri, mi) in enumerate(zip(roots, mults)):
        if used[i]:
            continue
        if sp.simplify(sp.im(ri)) == 0:
            reals.append((sp.nsimplify(ri), mi))
            used[i] = True
        else:
            ci = sp.conjugate(ri)
            j_found = None
            for j in range(i+1, len(roots)):
                if not used[j] and sp.simplify(roots[j] - ci) == 0 and mults[j] == mi:
                    j_found = j
                    break
            if j_found is None:
                fallback = True
                used[i] = True
            else:
                if sp.im(ri) < 0:
                    ri = ci
                pairs.append((sp.nsimplify(ri), mi))
                used[i] = True
                used[j_found] = True
    return reals, pairs, fallback

def real_hermite_system(A, roots, mults, mode, n=None, t=None):
    reals, pairs, fallback = split_real_conj(roots, mults)
    q = sum(mults)
    rows, rhs = [], []

    fact = sp.factorial
    ff   = sp.FallingFactorial

    for lam, r in reals:
        for j in range(r):
            row = [ (fact(k)//fact(k-j) * lam**(k-j) if k>=j else 0) for k in range(q) ]
            rows.append(row)
            if mode == 'power':
                rhs.append(sp.simplify(ff(n, j) * lam**(n - j)))
            elif mode == 'exp':
                rhs.append(sp.simplify((t**j) * sp.exp(t*lam)))
            elif mode == 'inv':
                rhs.append(0 if sp.simplify(lam)==0 else ((-1)**j)*fact(j)/lam**(j+1))
            else:
                raise ValueError("mode?")

    for z, rmult in pairs:
        rho, theta = sp.Abs(z), sp.arg(z)
        for j in range(rmult):
            rowR, rowI = [], []
            for k in range(q):
                if k < j:
                    rowR.append(0); rowI.append(0)
                else:
                    m = k - j
                    c = fact(k)//fact(k-j)
                    rowR.append(c * rho**m * sp.cos(m*theta))
                    rowI.append(c * rho**m * sp.sin(m*theta))
            rows += [rowR, rowI]
            if mode == 'power':
                p  = n - j
                coef = ff(n, j)
                rhs += [ sp.simplify(coef * rho**p * sp.cos(p*theta)),
                         sp.simplify(coef * rho**p * sp.sin(p*theta)) ]
            elif mode == 'exp':
                rhs += [ sp.simplify((t**j)*sp.exp(t*rho*sp.cos(theta))*sp.cos(t*rho*sp.sin(theta))),
                         sp.simplify((t**j)*sp.exp(t*rho*sp.cos(theta))*sp.sin(t*rho*sp.sin(theta))) ]
            elif mode == 'inv':
                p   = -(j+1)
                coef = ((-1)**j)*fact(j)
                rhs += [ sp.simplify(coef * rho**p * sp.cos(p*theta)),
                         sp.simplify(coef * rho**p * sp.sin(p*theta)) ]
            else:
                raise ValueError("mode?")

    M = sp.Matrix(rows)
    y = sp.Matrix(rhs)
    assert M.shape == (q, q)
    return M, y, fallback

def matfun_real_hermite(A, mode, n=None, t=None):
    A = sp.Matrix(A)
    mA = minimal_polynomial_via_linear(A)
    roots, mults = factor_minpoly(mA)
    M, y, _ = real_hermite_system(A, roots, mults, mode, n=n, t=t)
    coeffs = list(M.LUsolve(y))
    return evaluate_on_matrix(A, coeffs)

def power_via_CH_symbolic(A, n_symbol=None):
    n = n_symbol or sp.Symbol('n', integer=True)
    return sp.simplify(matfun_real_hermite(A, 'power', n=n))

def exp_via_CH_symbolic(A, t_symbol=None):
    t = t_symbol or sp.Symbol('t', real=True)
    return sp.simplify(matfun_real_hermite(A, 'exp', t=t))

def drazin_via_hermite(A):
    return sp.simplify(matfun_real_hermite(A, 'inv'))

## Cell 2: Symbolic demonstrations
We compute symbolic powers and exponentials for example matrices, as well as the Drazin inverse for a singular matrix. Results are displayed with LaTeX/MathJax.

In [2]:
import sympy as sp
from IPython.display import display, Math
sp.init_printing(use_latex='mathjax')

n = sp.Symbol('n', integer=True)
t = sp.Symbol('t', real=True)

# Example 1: 2x2 with complex spectrum ? A^n symbolic
A = sp.Matrix([[3, 2],
               [-1, 3]])
A_n = sp.simplify(power_via_CH_symbolic(A, n))
display(Math(r"A^n = " + sp.latex(A_n)))

EtA = sp.simplify(exp_via_CH_symbolic(A, t))
display(Math(r"\exp(tA) = " + sp.latex(EtA)))

# Example 2: 3x3 stochastic matrix
P = sp.Matrix([[0, 1, 0],
               [0, sp.Rational(2,3), sp.Rational(1,3)],
               [sp.Rational(1,3), 0, sp.Rational(2,3)]])
P_n = sp.simplify(power_via_CH_symbolic(P, n))
display(Math(r"P^n = " + sp.latex(P_n)))

E_tP = sp.simplify(exp_via_CH_symbolic(P, t))
display(Math(r"\exp(tP) = " + sp.latex(E_tP)))

# Example 3: Drazin inverse
C = sp.Matrix([[sp.Rational(1,5),  sp.Rational(2,5),  sp.Rational(2,5)],
               [sp.Rational(3,10), sp.Rational(3,5),  sp.Rational(1,10)],
               [sp.Rational(1,10), sp.Rational(1,5),  sp.Rational(7,10)]])
C_D = sp.simplify(drazin_via_hermite(C))
display(Math(r"C^D = " + sp.latex(C_D)))

# Sanity check: A^5
A5 = sp.simplify(A_n.subs(n, 5))
display(Math(r"A^5 = " + sp.latex(A5)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Cell 3: Minimal polynomial vs characteristic polynomial
We compare minimal and characteristic polynomials for several matrices, showing degrees and divisibility $m_A \mid \chi_A$.

In [3]:
import sympy as sp
x = sp.symbols('x')

def compare_matrix(A, name="A"):
    mA = minimal_polynomial_via_linear(A)
    charpoly = A.charpoly(x).as_poly().monic()
    print(f"\n{name}:")
    sp.pprint(A)
    print("m_A(x) =", mA.as_expr())
    print("?_A(x) =", charpoly.as_expr())
    print("Degrees: deg(m_A) =", mA.degree(), ", deg(?_A) =", charpoly.degree())
    print("Does m_A divide ?_A?", sp.rem(charpoly, mA, x) == 0)

# Example 1: Jordan block
J = sp.Matrix([[2,1,0],
               [0,2,1],
               [0,0,2]])
compare_matrix(J, "Jordan block 3x3 with eigenvalue 2")

# Example 2: Diagonal with distinct eigenvalues
D = sp.Matrix([[2,0],
               [0,3]])
compare_matrix(D, "Diagonal with eigenvalues 2,3")

# Example 3: Diagonal with repeated eigenvalue
B = sp.Matrix([[2,0,0],
               [0,2,0],
               [0,0,3]])
compare_matrix(B, "Diagonal with double eigenvalue 2 and one eigenvalue 3")

# Example 4: Singular non-diagonal matrix
C = sp.Matrix([[1,2],
               [0,0]])
compare_matrix(C, "Singular matrix C")


Jordan block 3x3 with eigenvalue 2:
⎡2  1  0⎤
⎢       ⎥
⎢0  2  1⎥
⎢       ⎥
⎣0  0  2⎦
m_A(x) = x**3 - 6*x**2 + 12*x - 8
?_A(x) = x**3 - 6*x**2 + 12*x - 8
Degrees: deg(m_A) = 3 , deg(?_A) = 3
Does m_A divide ?_A? True

Diagonal with eigenvalues 2,3:
⎡2  0⎤
⎢    ⎥
⎣0  3⎦
m_A(x) = x**2 - 5*x + 6
?_A(x) = x**2 - 5*x + 6
Degrees: deg(m_A) = 2 , deg(?_A) = 2
Does m_A divide ?_A? True

Diagonal with double eigenvalue 2 and one eigenvalue 3:
⎡2  0  0⎤
⎢       ⎥
⎢0  2  0⎥
⎢       ⎥
⎣0  0  3⎦
m_A(x) = x**2 - 5*x + 6
?_A(x) = x**3 - 7*x**2 + 16*x - 12
Degrees: deg(m_A) = 2 , deg(?_A) = 3
Does m_A divide ?_A? True

Singular matrix C:
⎡1  2⎤
⎢    ⎥
⎣0  0⎦
m_A(x) = x**2 - x
?_A(x) = x**2 - x
Degrees: deg(m_A) = 2 , deg(?_A) = 2
Does m_A divide ?_A? True
